## Project solution

In [ ]:
from typing import List, Tuple, Union
import imageio

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def add_padding(image: np.ndarray, padding: Tuple[int, int]) -> np.ndarray:
    height, width = image.shape
    pad_height, pad_width = padding

    padded_image = np.zeros((height + pad_height * 2, width + pad_width * 2))
    padded_image[
        pad_height : height + pad_height, pad_width : width + pad_width
    ] = image

    return padded_image


def check_params(
    image: np.ndarray, kernel: np.ndarray, padding: Tuple[int, int] = (2, 2)
):
    params_are_correct = (
        isinstance(padding[0], int)
        and isinstance(padding[1], int)
        and padding[0] >= 0
        and padding[1] >= 0
    )
    assert params_are_correct, "padding values have to be positive integers"
    height, width = image.shape
    image = image if list(padding) == [0, 0] else add_padding(image, padding)
    height_padded, width_padded = image.shape
    kernel_shape = kernel.shape

    kernel_is_correct = kernel_shape[0] % 2 == 1 and kernel_shape[1] % 2 == 1
    assert kernel_is_correct, "Kernel shape has to be odd."
    image_to_kernel_is_correct = (
        height_padded >= kernel_shape[0] and width_padded >= kernel_shape[1]
    )
    assert image_to_kernel_is_correct, "Kernel has to be smaller than image"

    h_out = (
        np.floor(
            (height + 2 * padding[0] - kernel_shape[0] - (kernel_shape[0] - 1))
        ).astype(int)
        + 1
    )
    w_out = (
        np.floor(
            (width + 2 * padding[1] - kernel_shape[1] - (kernel_shape[1] - 1))
        ).astype(int)
        + 1
    )
    out_dimensions_are_correct = h_out > 0 and w_out > 0
    assert out_dimensions_are_correct

    return image, kernel, kernel_shape, h_out, w_out


def convolve2D(
    image: np.ndarray, kernel: np.ndarray, padding: Tuple[int, int] = (2, 2)
) -> np.ndarray:
    image, kernel, kernel_shape, h_out, w_out = check_params(image, kernel, padding)
    image_out = np.zeros((h_out, w_out))

    center_pixels = kernel_shape[0] // 2, kernel_shape[1] // 2
    center_x_0 = center_pixels[0]
    center_y_0 = center_pixels[1]
    for i in range(h_out):
        center_x = center_x_0 + i
        indices_x = [
            center_x + l for l in range(-center_pixels[0], center_pixels[0] + 1)
        ]
        for j in range(w_out):
            center_y = center_y_0 + j
            indices_y = [
                center_y + l for l in range(-center_pixels[1], center_pixels[1] + 1)
            ]

            crop_image = image[indices_x, :][:, indices_y]

            image_out[i][j] = np.sum(np.multiply(crop_image, kernel))
    return image_out


def apply_filter(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    b = kernel.shape
    return np.dstack(
        [
            convolve2D(image[:, :, z], kernel, padding=(b[0] // 2, b[1] // 2))
            for z in range(3)
        ]
    ).astype("uint8")

In [ ]:
image = imageio.imread("example.png")
plt.imshow(image)

In [ ]:
gaussian_blur = (
    np.array(
        [
            [1, 4, 6, 4, 1],
            [4, 16, 24, 16, 4],
            [6, 24, 36, 24, 6],
            [4, 16, 24, 16, 4],
            [1, 4, 6, 4, 1],
        ]
    )
    / 256
)

In [ ]:
filtered_image = apply_filter(image, gaussian_blur)
plt.imshow(filtered_image, vmin=0, vmax=255)